In [1]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib.cm as cm
from matplotlib import gridspec
from scipy import signal

import numpy as np
import pandas as pd
import os
import shutil
import cv2
from tqdm import trange

#import glob2

from numba import jit
import tables
from scipy.io import loadmat
import scipy
import h5py
#import hdf5storage
import csv

from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components


Autosaving every 180 seconds


In [2]:

import os
os.environ["DLClight"] = "True"
import deeplabcut
import tensorflow.contrib.slim as slim
from pathlib import Path
import numpy as np
import tensorflow as tf
vers = (tf.__version__).split(".")
if int(vers[0]) == 1 and int(vers[1]) > 12:
    TF = tf.compat.v1
else:
    TF = tf
    
import tensorflow as tf
from tensorflow.contrib.slim.nets import resnet_v1
from tensorflow.keras.datasets import cifar10
#from deeplabcut.pose_estimation_tensorflow.train import get_optimizer, LearningRate
from deeplabcut.pose_estimation_tensorflow.train import  LearningRate


/home/cat/.conda/envs/DLC-GPU/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/cat/.conda/envs/DLC-GPU/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/cat/.conda/envs/DLC-GPU/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/cat/.conda/envs/DLC-GPU/lib/python3.7/site-packages/ten

DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


In [3]:

def get_optimizer(loss_op, cfg, train_resnet=False):
    # train_resnet=True trains resnet!
    learning_rate = TF.placeholder(tf.float32, shape=[])

    if cfg.optimizer == "sgd":
        optimizer = TF.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
    elif cfg.optimizer == "adam":
        optimizer = TF.train.AdamOptimizer(learning_rate)
    else:
        raise ValueError('unknown optimizer {}'.format(cfg.optimizer))

    #%%
    all_variables_to_train = tf.trainable_variables()
    if train_resnet:
        variables_to_train = all_variables_to_train
    else:
        variables_to_train = list(filter(lambda k: 'mclassid' in k.name, all_variables_to_train))

    train_op = slim.learning.create_train_op(loss_op, optimizer, variables_to_train=variables_to_train)

    return learning_rate, train_op


def get_train_config(cfg, shuffle=1):
    from deeplabcut.utils import auxiliaryfunctions
    from deeplabcut.pose_estimation_tensorflow.config import load_config
    project_path = cfg['project_path']
    iteration = cfg['iteration']
    TrainingFraction = cfg['TrainingFraction'][iteration]
    modelfolder = os.path.join(
        project_path,
        str(auxiliaryfunctions.GetModelFolder(TrainingFraction, shuffle, cfg)))

    path_test_config = Path(modelfolder) / 'train' / 'pose_cfg.yaml'
    print(path_test_config)
    try:
        dlc_cfg = load_config(str(path_test_config))
    except FileNotFoundError:
        raise FileNotFoundError(
            "It seems the model for shuffle %s and trainFraction %s does not exist."
            % (shuffle, TrainingFraction))
    return dlc_cfg

In [14]:
#
# ------------------- User inputs --------

import tensorflow as tf
from tensorflow.contrib.slim.nets import resnet_v1
from tensorflow.keras.datasets import cifar10


# define paths
config_path = '/media/cat/4TBSSD/DLC_full_directory/config.yaml'
videofile_path = '/media/cat/4TBSSD/DLC_full_directory/videos_to_label/2020-3-16_12_54_07_193951_compressed_label.mp4'
shuffle = 1 #dlc experiment shuffle

# ------------------- READ DLC info --------
# read config
print("Reading config...")
cfg = deeplabcut.auxiliaryfunctions.read_config(str(config_path))
dlc_cfg = get_train_config(cfg,shuffle=shuffle)

#

#
# train w imagenet: data/255
BATCH_SIZE = 60
num_classes = 10
is_training=False

#%%
# Example with CIFAR10, replace with frames and labels
#(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# CIFAR DATA: x_train (50000, 32, 32, 3) 
#             y_train (50000, 1) 
#             x_test (10000, 32, 32, 3) 
#             y_test (10000, 1)

def load_gerbil_training_data(animal_dir):
    
    from skimage.measure import block_reduce
    
    x_train = []
    y_train = []
    x_test = []
    y_test = []
    for k in range(4):
        data = np.load(animal_dir+"id_"+str(k)+'.npy')
        
        # scale data down:
        #image = np.arange(3*3*4).reshape(3, 3, 4)
        data_downsampled = []
        for p in range(data.shape[0]):
            temp = data[k]
           # print ("temp start: ", temp.shape)
            temp = block_reduce(data[k], 
                                block_size=(6,6), 
                                func=np.mean)
            #print ("temp finish: ", temp.shape)
            data_downsampled.append(temp)
        data = np.array(data_downsampled)
        
        idx_rand = np.random.choice(np.arange(data.shape[0]), 500, replace=False)
        #print ("idx rand: ", idx_rand.shape)
        
        x_train.append(data[idx_rand])
        print (data[idx_rand].shape)
        y_train.append(np.zeros(idx_rand.shape[0])+k)

        # make test set
        idx_test = np.delete(np.arange(data.shape[0]), 
                             idx_rand)
        #print ("idx test: ", idx_test.shape)
        temp = data[idx_test]
        #print (temp.shape)
        x_test.append(temp)
        y_test.append(np.zeros(temp.shape[0])+k)

        

    x_train = np.vstack(x_train)
    x_train = np.int32((x_train,x_train,x_train)).transpose(1,2,3,0)
    #np.save('/home/cat/x_train.npy', x_train)
    
    y_train = np.hstack(y_train)[:,None]
    
    x_test = np.vstack(x_test)
    x_test = np.int32((x_test,x_test,x_test)).transpose(1,2,3,0)
    y_test = np.hstack(y_test)[:,None]
        
        
    return x_train, y_train, x_test, y_test 

animal_dir ='/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/training_images/animals/'
(x_train, y_train, x_test, y_test) = load_gerbil_training_data(animal_dir)


print ("x train: ", x_train.shape, 
       "y train: ", y_train.shape, 
       "x test: ", x_test.shape, 
       "y test: ", y_test.shape)

# for k in range(10):
#     ax=plt.subplot(2,5,k+1)
#     plt.imshow(x_train[k])
# plt.show()

Reading config...
/media/cat/4TBSSD/DLC_full_directory/dlc-models/iteration-0/madeline_july2Jul2-trainset95shuffle1/train/pose_cfg.yaml
(500, 34, 34)
(500, 34, 34)
(500, 34, 34)
(500, 34, 34)
x train:  (2000, 34, 34, 3) y train:  (2000, 1) x test:  (380, 34, 34, 3) y test:  (380, 1)


In [15]:

x_train, x_test = np.array(x_train,np.float32), np.array(x_test, np.float32)
y_train, y_test = np.array(y_train.flatten(), np.int64), np.array(y_test.flatten(), np.int64)

nx_in, ny_in, nc_in = x_train.shape[1:]
assert nc_in == 3

print ("X_train: ", x_train.shape)
np.save('/home/cat/x_train.npy', x_train)

X_train:  (2000, 34, 34, 3)


In [16]:
# plt.imshow(x_train[0])
# plt.show()

In [17]:

#%% ------------- BEGIN GRAPH -------------
TF.reset_default_graph()
# tf.data API
batch_size = tf.placeholder(tf.int64, shape=[])
x, y = tf.placeholder(tf.float32, shape=[None, nx_in, ny_in, nc_in]), tf.placeholder(tf.int32, shape=[None,])
train_dataset = tf.data.Dataset.from_tensor_slices((x, y)).batch(batch_size).repeat().shuffle(5000)
test_dataset = tf.data.Dataset.from_tensor_slices((x, y)).batch(batch_size).shuffle(5000)
iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                           train_dataset.output_shapes)
image_batch, targets = iterator.get_next()


print ("train_dataset: ", train_dataset, image_batch.shape)

train_dataset:  <DatasetV1Adapter shapes: ((?, 34, 34, 3), (?,)), types: (tf.float32, tf.int32)> (?, 34, 34, 3)


In [18]:

# init iterators
train_init_op = iterator.make_initializer(train_dataset)
test_init_op = iterator.make_initializer(test_dataset)

# upsample cifar images which are 32 x 32
if nx_in <= 200:
    inputs = tf.image.resize_images(image_batch, [224, 224])
print (inputs.shape)
targets = tf.reshape(targets, [batch_size, ])

# Make model
net_funcs = {'resnet_50': resnet_v1.resnet_v1_50,
             'resnet_101': resnet_v1.resnet_v1_101,
             'resnet_152': resnet_v1.resnet_v1_152}

net_fun = net_funcs[dlc_cfg.net_type]



(?, 224, 224, 3)


In [19]:
# preprocess the same way as dlc -- here note that they don't normalize
mean = tf.constant(dlc_cfg.mean_pixel, dtype=tf.float32, shape=[1, 1, 1, 3],
                   name='img_mean')
im_centered = inputs - mean
# add strides st we can load dlc weights
if 'output_stride' not in dlc_cfg.keys():
    dlc_cfg.output_stride = 16
if 'deconvolutionstride' not in dlc_cfg.keys():
    dlc_cfg.deconvolutionstride = 2
    
# 
with slim.arg_scope(resnet_v1.resnet_arg_scope()):
    net, endpoints = net_fun(im_centered, num_classes=None,#num_classes,
                             is_training=is_training,
                             #global_pool=True,
                             output_stride=dlc_cfg.output_stride)

net = tf.squeeze(net, axis=[1, 2])
logits = slim.fully_connected(net, num_outputs=num_classes,
                              activation_fn=None,
                              scope='mclassid')
loss = tf.reduce_mean(
    tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=targets))



In [20]:

# add extrainfo
logits2 = tf.nn.softmax(logits)
classes = tf.cast(tf.argmax(logits2, axis=1, ), tf.int32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(classes, targets), dtype=tf.float32))
eval_dict = {'logits2': logits2,
             'classes': classes,
             'accuracy':accuracy}

# restore variables for resnet
variables_to_restore = slim.get_variables_to_restore(include=["resnet_v1"])
restorer = TF.train.Saver(variables_to_restore)
saver = TF.train.Saver(
        max_to_keep=5
    )



In [21]:
# %% Init session
#ess.close()
config_TF = TF.ConfigProto()
config_TF.gpu_options.allow_growth = True
sess = TF.Session(config=config_TF)
TF.summary.FileWriter('logs/', sess.graph)

learning_rate, train_op = get_optimizer(loss, dlc_cfg)
lr = tf.get_variable('learning_rate', initializer=0.01, trainable=False)
#%%
sess.run(TF.global_variables_initializer())
sess.run(TF.local_variables_initializer())
#sess.save()

# Restore the resnet weights from dlc
print ("Previous dlc_cfg init weights: ", dlc_cfg.init_weights)
dlc_cfg.init_weights = '/home/cat/Downloads/resnet_v1_50.ckpt'
restorer.restore(sess, dlc_cfg.init_weights)
print('Restored variables from\n{}\n'.format(dlc_cfg.init_weights))


Previous dlc_cfg init weights:  C:\ProgramData\Anaconda3\envs\DLC-GPU2\lib\site-packages\deeplabcut\pose_estimation_tensorflow\models\pretrained\resnet_v1_50.ckpt
Restored variables from
/home/cat/Downloads/resnet_v1_50.ckpt



In [22]:
# TRAIN NETWORK
print (y_train.shape, y_train)
print (x_train.shape)
if True:
    lr_gen = LearningRate(dlc_cfg)
    #%%
    EPOCHS = 10 # 10000
    current_lr = 0.0001
    n_batches = max(1, int(x_train.shape[0] / BATCH_SIZE))
    #n_batches = 30
    print('Training...')
    for epoch in range(EPOCHS):
        sess.run(train_init_op, feed_dict={x: x_train, 
                                           y: y_train, 
                                           batch_size: BATCH_SIZE,
                                           learning_rate: current_lr})

        tot_loss = 0
        for iter in range(n_batches):
            _, loss_value, accuracy_value = sess.run([train_op, loss, accuracy],
                                                     feed_dict={learning_rate:current_lr})
            print("Epoch: {} \t Iter: {}/{}, Loss: {:.4f} Accuracy:{:.4f}".format(epoch, iter, n_batches, loss_value, accuracy_value))
            tot_loss += loss_value

            if iter % 100 == 0 or (iter+1 == n_batches):
                model_name = 'snapshots/mclass_epoch{}-iter{}-'.format(epoch,iter)
                saver.save(sess, model_name, global_step=iter)
                if iter +1  == EPOCHS:
                    model_name = 'snapshots/mclass_epoch{}-iter{}-final-'.format(epoch,iter)
                    saver.save(sess, model_name, global_step=0)

        print("Epoch: {}, Loss: {:.4f}".format(epoch, tot_loss / n_batches))
        
        # initialise iterator with test data
        sess.run(test_init_op,
                 feed_dict={x: x_test, 
                            y: y_test, 
                            batch_size: BATCH_SIZE, 
                            learning_rate:current_lr})
        
        print('Epoch: {}, '.format(epoch) + 'Test Loss: {:.4f} Test Accuracy {:.4f}'.format(*sess.run([loss,accuracy])))


#sess.close()



(2000,) [0 0 0 ... 3 3 3]
(2000, 34, 34, 3)
Training...
Epoch: 0 	 Iter: 0/33, Loss: 2.1462 Accuracy:0.0000
Epoch: 0 	 Iter: 1/33, Loss: 2.5226 Accuracy:0.0000
Epoch: 0 	 Iter: 2/33, Loss: 2.4110 Accuracy:0.0000
Epoch: 0 	 Iter: 3/33, Loss: 2.2843 Accuracy:0.0000
Epoch: 0 	 Iter: 4/33, Loss: 1.9533 Accuracy:0.0000
Epoch: 0 	 Iter: 5/33, Loss: 1.8757 Accuracy:0.0000
Epoch: 0 	 Iter: 6/33, Loss: 1.7340 Accuracy:0.0000
Epoch: 0 	 Iter: 7/33, Loss: 2.3452 Accuracy:0.0000
Epoch: 0 	 Iter: 8/33, Loss: 3.2127 Accuracy:0.0000
Epoch: 0 	 Iter: 9/33, Loss: 3.1833 Accuracy:0.0000
Epoch: 0 	 Iter: 10/33, Loss: 1.5377 Accuracy:1.0000
Epoch: 0 	 Iter: 11/33, Loss: 3.0968 Accuracy:0.0000
Epoch: 0 	 Iter: 12/33, Loss: 2.1737 Accuracy:0.0000
Epoch: 0 	 Iter: 13/33, Loss: 2.9981 Accuracy:0.0000
Epoch: 0 	 Iter: 14/33, Loss: 2.0751 Accuracy:0.0000
Epoch: 0 	 Iter: 15/33, Loss: 1.3953 Accuracy:1.0000
Epoch: 0 	 Iter: 16/33, Loss: 1.9549 Accuracy:0.0000
Epoch: 0 	 Iter: 17/33, Loss: 1.8852 Accuracy:0.0000


Epoch: 4 	 Iter: 18/33, Loss: 0.5096 Accuracy:1.0000
Epoch: 4 	 Iter: 19/33, Loss: 0.1550 Accuracy:1.0000
Epoch: 4 	 Iter: 20/33, Loss: 0.4807 Accuracy:1.0000
Epoch: 4 	 Iter: 21/33, Loss: 0.1528 Accuracy:1.0000
Epoch: 4 	 Iter: 22/33, Loss: 0.1511 Accuracy:1.0000
Epoch: 4 	 Iter: 23/33, Loss: 0.1485 Accuracy:1.0000
Epoch: 4 	 Iter: 24/33, Loss: 0.4309 Accuracy:1.0000
Epoch: 4 	 Iter: 25/33, Loss: 0.4193 Accuracy:1.0000
Epoch: 4 	 Iter: 26/33, Loss: 0.1328 Accuracy:1.0000
Epoch: 4 	 Iter: 27/33, Loss: 0.5853 Accuracy:1.0000
Epoch: 4 	 Iter: 28/33, Loss: 0.3861 Accuracy:1.0000
Epoch: 4 	 Iter: 29/33, Loss: 0.5830 Accuracy:1.0000
Epoch: 4 	 Iter: 30/33, Loss: 0.1340 Accuracy:1.0000
Epoch: 4 	 Iter: 31/33, Loss: 0.1339 Accuracy:1.0000
Epoch: 4 	 Iter: 32/33, Loss: 0.1355 Accuracy:1.0000
Epoch: 4, Loss: 0.3121
Epoch: 4, Test Loss: 0.5619 Test Accuracy 1.0000
Epoch: 5 	 Iter: 0/33, Loss: 0.1345 Accuracy:1.0000
Epoch: 5 	 Iter: 1/33, Loss: 0.1328 Accuracy:1.0000
Epoch: 5 	 Iter: 2/33, Loss: 

Epoch: 9 	 Iter: 2/33, Loss: 0.2131 Accuracy:1.0000
Epoch: 9 	 Iter: 3/33, Loss: 0.0638 Accuracy:1.0000
Epoch: 9 	 Iter: 4/33, Loss: 0.0632 Accuracy:1.0000
Epoch: 9 	 Iter: 5/33, Loss: 0.1022 Accuracy:1.0000
Epoch: 9 	 Iter: 6/33, Loss: 0.0617 Accuracy:1.0000
Epoch: 9 	 Iter: 7/33, Loss: 0.0608 Accuracy:1.0000
Epoch: 9 	 Iter: 8/33, Loss: 0.2068 Accuracy:1.0000
Epoch: 9 	 Iter: 9/33, Loss: 0.1004 Accuracy:1.0000
Epoch: 9 	 Iter: 10/33, Loss: 0.0999 Accuracy:1.0000
Epoch: 9 	 Iter: 11/33, Loss: 0.0576 Accuracy:1.0000
Epoch: 9 	 Iter: 12/33, Loss: 0.0569 Accuracy:1.0000
Epoch: 9 	 Iter: 13/33, Loss: 0.0560 Accuracy:1.0000
Epoch: 9 	 Iter: 14/33, Loss: 0.0978 Accuracy:1.0000
Epoch: 9 	 Iter: 15/33, Loss: 0.0541 Accuracy:1.0000
Epoch: 9 	 Iter: 16/33, Loss: 0.0658 Accuracy:1.0000
Epoch: 9 	 Iter: 17/33, Loss: 0.1980 Accuracy:1.0000
Epoch: 9 	 Iter: 18/33, Loss: 0.0516 Accuracy:1.0000
Epoch: 9 	 Iter: 19/33, Loss: 0.0658 Accuracy:1.0000
Epoch: 9 	 Iter: 20/33, Loss: 0.1937 Accuracy:1.0000
E

In [50]:
model_path = "/home/cat/code/dlc_toolbox/snapshots/mclass_epoch1-iter32--32"
detection_graph = tf.Graph()
with tf.Session(graph=detection_graph) as sess:
    # Load the graph with the trained states
    loader = tf.train.import_meta_graph(model_path+'.meta')
    loader.restore(sess, model_path)
    
    current_graph = tf.compat.v1.get_default_graph()
    #for k in current_graph.get_operations():
    #    print(k)    

    # Get the tensors by their variable name
    #image_tensor = detection_graph.get_tensor_by_name(':0')
    
    boxes = detection_graph.get_tensor_by_name('accuracy:0')
    #scores = detection_graph.get_tensor_by_name('logits2:0')
    #...
    # Make predictions
    #_boxes, _scores = sess.run([boxes, scores], 
    #                           feed_dict={image_tensor: image_np_expanded})


KeyError: "The name 'accuracy:0' refers to a Tensor which does not exist. The operation, 'accuracy', does not exist in the graph."

In [58]:
model_path = "model.ckpt"
detection_graph = tf.Graph()
with tf.Session(graph=detection_graph) as sess:
    # Load the graph with the trained states
    loader = tf.train.import_meta_graph(model_path+'.meta')
    loader.restore(sess, model_path)

    # Get the tensors by their variable name
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
    boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
    scores = detection_graph.get_tensor_by_name('detection_scores:0')
    ...
    # Make predictions
    _boxes, _scores = sess.run([boxes, scores], feed_dict={image_tensor: image_np_expanded})

test:  name: "make_initializer_1"
op: "MakeIterator"
input: "ShuffleDataset_1"
input: "IteratorV2"
device: "/device:CPU:0"
attr {
  key: "_class"
  value {
    list {
      s: "loc:@IteratorV2"
    }
  }
}



RuntimeError: Attempted to use a closed Session.

In [30]:
res = sess.predict(test_init_op,  
               feed_dict = feed_dict)



print (out['classes'].shape, out['classes'])

AttributeError: 'Session' object has no attribute 'predict'

In [38]:
# 

#image_batch = x_test[:10]
#inputs = tf.image.resize_images(image_batch, [224, 224])
#print (inputs.shape)
graph = tf.get_default_graph()
    
feed_dict={x: x_test[:10]}

##model = graph.get_tensor_by_name('resnet_model/final_dense:0')
#model.predict()
model = graph.get_tensor_by_name('/home/cat/code/dlc_toolbox/snapshots/mclass_epoch1-iter32--32.data-00000-of-00001')

#res = sess.run(model, {inputs:img})
    
#res = sess.predict(test_init_op,  
#               feed_dict = feed_dict)




ValueError: The name '/home/cat/code/dlc_toolbox/snapshots/mclass_epoch1-iter32--32.data-00000-of-00001' looks like an (invalid) Operation name, not a Tensor. Tensor names must be of the form "<op_name>:<output_index>".

In [ ]:
with tf.Session(graph=tf.Graph()) as sess:
    tf.saved_model.loader.load(sess,["serve"], graph_pb_path)
    graph = tf.get_default_graph()
    inputs = graph.get_tensor_by_name('input_tensor:0')


        

In [25]:
# pn = pose_net(dlc_cfg)

# net, end_points = pn.extract_features(inputs)
# scope = "pose"
# reuse = None
# heads = {}

# with tf.variable_scope(scope, reuse=reuse):
#     heads["part_pred"] = prediction_layer(
#         dlc_cfg, net, "part_pred", nj
#     )
#     heads["locref"] = prediction_layer(
#         dlc_cfg, net, "locref_pred", nj * 2
#     )

#     heads["pairwise_pred"] = prediction_layer(
#         dlc_cfg, net, "pairwise_pred", nl * 2
#     )
# pred = heads['part_pred']

NameError: name 'pose_net' is not defined

In [23]:
print ("xtest: ", x_test.shape)
pred = heads['part_pred']

feed_dict = {inputs: x_test[0]}
res = sess.run([pred], feed_dict=feed_dict)

print (res)

xtest:  (380, 34, 34, 3)


NameError: name 'heads' is not defined

In [ ]:

# feed to the network
feed_dict = {inputs: ff[None, :, :, :]}
[pred_s] = sess.run([pred], feed_dict=feed_dict)

In [23]:
print (x_test.shape,
      y_test.shape,
      BATCH_SIZE,
      current_lr)
print (y_test)

(10000, 32, 32, 3) (10000,) 60 0.0001
[3 8 8 ... 5 1 7]
